# Multiple Layer LSTM

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


In [ ]:
tokenizer = info.features['text'].encoder

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

```
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))
```
* LSTM layer : `tf.keras.layers.LSTM(units=)`
  * LSTM에 전달된 파라미터는 units이다. 즉, 예시 코드의 경우 Embedding layer의 output_dim이 64이므로, LSTM 레이어의 유닛으로 64가 똑같이 전달되어야 하는 것.
* Bidirectional layer
  * LSTM layer를 감싸야 한다(wrap up)
  * 이 때 Bidirectional 레이어로 감싸면 cell state를 양방향(both direction)으로 만든다.
  * 따라서 `model.summary`를 해 보면 64개 units을 bidirection으로 전달했기 때문에 Output shape은 128로 출력된다!

### Multi layer LSTM
```
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64), return_sequences=True),
tf.keras.layers.Bidirectioanl(tf.keras.layers.LSTM(32))
```
멀티레이어의 경우 앞의 레이어에 `return_sequences=True`를 설정해라(default=False). 이 파라미터는 LSTM의 아웃풋이 다음 레이어의 원하는 입력과 일치하게 한다.


### LSTM Layer APIs
```
tf.keras.layers.LSTM(
    units, activation='tanh', recurrent_activation='sigmoid',
    use_bias=True, kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal',
    bias_initializer='zeros', unit_forget_bias=True,
    kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
    activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None,
    bias_constraint=None, dropout=0.0, recurrent_dropout=0.0,
    return_sequences=False, return_state=False, go_backwards=False, stateful=False,
    time_major=False, unroll=False, **kwargs
)
```



In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')